In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_digits, make_swiss_roll, make_s_curve
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import trustworthiness
import warnings
warnings.filterwarnings('ignore')

# Set style and random seed
plt.style.use('seaborn-v0_8')
np.random.seed(42)

print("Libraries imported successfully!")


In [ ]:
class AdvancedTSNE:
    """Advanced t-SNE implementation with detailed analysis capabilities"""
    
    def __init__(self, n_components=2, perplexity=30, learning_rate=200, 
                 n_iter=1000, early_exaggeration=12, min_grad_norm=1e-7):
        self.n_components = n_components
        self.perplexity = perplexity
        self.learning_rate = learning_rate
        self.n_iter = n_iter
        self.early_exaggeration = early_exaggeration
        self.min_grad_norm = min_grad_norm
        
        # Training history
        self.costs = []
        self.grad_norms = []
        self.embedding_history = []
        self.P_matrix = None
        
    def _pairwise_distances(self, X):
        """Compute pairwise squared Euclidean distances efficiently"""
        sum_X = np.sum(X**2, axis=1)
        distances = sum_X[:, np.newaxis] + sum_X[np.newaxis, :] - 2 * np.dot(X, X.T)
        return np.maximum(distances, 0)
    
    def _search_optimal_sigma(self, distances_i, target_perplexity, tol=1e-5, max_iter=50):
        """Binary search for optimal sigma for given perplexity"""
        beta_min = -np.inf
        beta_max = np.inf
        beta = 1.0
        
        for _ in range(max_iter):
            # Compute conditional probabilities
            Pi = np.exp(-beta * distances_i)
            sum_Pi = np.sum(Pi)
            
            if sum_Pi == 0:
                Pi = np.ones_like(Pi) / len(Pi)
                sum_Pi = 1.0
            else:
                Pi = Pi / sum_Pi
            
            # Compute perplexity
            H = -np.sum(Pi * np.log2(Pi + 1e-12))
            perplexity = 2**H
            
            # Check convergence
            perp_diff = perplexity - target_perplexity
            if abs(perp_diff) < tol:
                break
            
            # Update beta
            if perp_diff > 0:
                beta_min = beta
                if beta_max == np.inf:
                    beta = beta * 2
                else:
                    beta = (beta + beta_max) / 2
            else:
                beta_max = beta
                if beta_min == -np.inf:
                    beta = beta / 2
                else:
                    beta = (beta + beta_min) / 2
        
        return beta
    
    def _compute_joint_probabilities(self, X):
        """Compute joint probabilities in high-dimensional space with detailed logging"""
        n = X.shape[0]
        distances = self._pairwise_distances(X)
        
        # Set diagonal to infinity to exclude self-distances
        np.fill_diagonal(distances, np.inf)
        
        # Compute conditional probabilities for each point
        P = np.zeros((n, n))
        sigmas = np.zeros(n)
        
        print("Computing optimal sigmas for each point...")
        for i in range(n):
            if i % 100 == 0:
                print(f"Processing point {i}/{n}")
            
            # Binary search for optimal sigma
            beta = self._search_optimal_sigma(distances[i], self.perplexity)
            sigmas[i] = 1.0 / np.sqrt(2.0 * beta)
            
            # Compute conditional probabilities
            Pi = np.exp(-beta * distances[i])
            Pi[i] = 0  # Exclude self
            Pi = Pi / np.sum(Pi)
            P[i] = Pi
        
        # Symmetrize to get joint probabilities
        P_joint = (P + P.T) / (2.0 * n)
        P_joint = np.maximum(P_joint, 1e-12)
        
        print(f"Sigma statistics: mean={np.mean(sigmas):.4f}, std={np.std(sigmas):.4f}")
        print(f"Perplexity range achieved: {2**(-np.sum(P * np.log2(P + 1e-12), axis=1)).min():.2f} - {2**(-np.sum(P * np.log2(P + 1e-12), axis=1)).max():.2f}")
        
        return P_joint, sigmas
    
    def _compute_low_dim_probabilities(self, Y):
        """Compute probabilities in low-dimensional space using Student's t-distribution"""
        distances = self._pairwise_distances(Y)
        
        # Student's t-distribution with 1 degree of freedom
        Q = 1.0 / (1.0 + distances)
        np.fill_diagonal(Q, 0)
        
        # Normalize
        Q = Q / np.sum(Q)
        Q = np.maximum(Q, 1e-12)
        
        return Q
    
    def _compute_gradient(self, P, Q, Y):
        """Compute gradient with detailed force analysis"""
        n = Y.shape[0]
        
        # Attractive and repulsive forces
        PQ_diff = P - Q
        distances = self._pairwise_distances(Y)
        inv_distances = 1.0 / (1.0 + distances)
        
        # Compute gradient
        gradient = np.zeros_like(Y)
        
        for i in range(n):
            diff = Y[i] - Y
            forces = (PQ_diff[i] * inv_distances[i])[:, np.newaxis] * diff
            gradient[i] = 4 * np.sum(forces, axis=0)
        
        return gradient
    
    def _analyze_forces(self, P, Q, Y, iteration):
        """Analyze attractive vs repulsive forces"""
        n = Y.shape[0]
        distances = self._pairwise_distances(Y)
        
        # Compute forces
        attractive_force = np.sum(P * distances)
        repulsive_force = np.sum(Q * distances)
        
        # Compute KL divergence components
        kl_div = np.sum(P * np.log(P / Q))
        
        if iteration % 100 == 0:
            print(f"Iteration {iteration}: KL={kl_div:.4f}, Attractive={attractive_force:.4f}, Repulsive={repulsive_force:.4f}")
        
        return {
            'kl_divergence': kl_div,
            'attractive_force': attractive_force,
            'repulsive_force': repulsive_force
        }
    
    def fit_transform(self, X):
        """Fit t-SNE with early exaggeration and detailed monitoring"""
        n, d = X.shape
        print(f"Fitting t-SNE on {n} points with {d} dimensions...")
        
        # Compute joint probabilities
        P, sigmas = self._compute_joint_probabilities(X)
        self.P_matrix = P.copy()
        
        # Initialize embedding
        Y = np.random.normal(0, 1e-4, (n, self.n_components))
        
        # Momentum terms for optimization
        Y_momentum = np.zeros_like(Y)
        
        print("Starting optimization...")
        for iteration in range(self.n_iter):
            # Apply early exaggeration
            if iteration < 250:
                P_current = P * self.early_exaggeration
            else:
                P_current = P.copy()
            
            # Compute low-dimensional probabilities
            Q = self._compute_low_dim_probabilities(Y)
            
            # Analyze forces
            force_analysis = self._analyze_forces(P_current, Q, Y, iteration)
            
            # Compute cost (KL divergence)
            cost = np.sum(P_current * np.log(P_current / Q))
            self.costs.append(cost)
            
            # Compute gradient
            gradient = self._compute_gradient(P_current, Q, Y)
            grad_norm = np.linalg.norm(gradient)
            self.grad_norms.append(grad_norm)
            
            # Early stopping if gradient is too small
            if grad_norm < self.min_grad_norm:
                print(f"Stopping early at iteration {iteration} due to small gradient norm")
                break
            
            # Update with momentum
            if iteration < 250:
                momentum = 0.5
                lr = self.learning_rate
            else:
                momentum = 0.8
                lr = self.learning_rate
            
            Y_momentum = momentum * Y_momentum - lr * gradient
            Y = Y + Y_momentum
            
            # Center the embedding
            Y = Y - np.mean(Y, axis=0)
            
            # Store embedding history (every 50 iterations to save memory)
            if iteration % 50 == 0:
                self.embedding_history.append(Y.copy())
            
        print(f"Optimization completed after {len(self.costs)} iterations")
        return Y

# Analyze perplexity effects
def analyze_perplexity_effects():
    """Analyze how different perplexity values affect t-SNE embeddings"""
    
    # Load digits dataset for analysis
    digits = load_digits()
    X = digits.data
    y = digits.target
    
    # Use subset for faster computation
    n_samples = 500
    indices = np.random.choice(len(X), n_samples, replace=False)
    X_subset = X[indices]
    y_subset = y[indices]
    
    # Standardize
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_subset)
    
    # Test different perplexity values
    perplexities = [5, 15, 30, 50, 100]
    
    results = {}
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for i, perp in enumerate(perplexities):
        print(f"\\nTesting perplexity = {perp}")
        
        # Fit t-SNE
        tsne = AdvancedTSNE(perplexity=perp, n_iter=500, learning_rate=200)
        Y = tsne.fit_transform(X_scaled)
        
        # Calculate trustworthiness
        trust_score = trustworthiness(X_scaled, Y, n_neighbors=min(10, perp//2))
        
        results[perp] = {
            'embedding': Y,
            'trustworthiness': trust_score,
            'final_cost': tsne.costs[-1],
            'tsne': tsne
        }
        
        # Plot embedding
        scatter = axes[i].scatter(Y[:, 0], Y[:, 1], c=y_subset, cmap='tab10', alpha=0.7)
        axes[i].set_title(f'Perplexity = {perp}\\nTrust: {trust_score:.3f}')
        axes[i].set_xlabel('t-SNE 1')
        axes[i].set_ylabel('t-SNE 2')
    
    # Plot perplexity comparison
    axes[5].set_visible(False)
    
    # Create comparison plot
    fig2, axes2 = plt.subplots(1, 3, figsize=(18, 5))
    
    # Trustworthiness vs perplexity
    perps = list(results.keys())
    trusts = [results[p]['trustworthiness'] for p in perps]
    costs = [results[p]['final_cost'] for p in perps]
    
    axes2[0].plot(perps, trusts, 'o-', linewidth=2, markersize=8)
    axes2[0].set_xlabel('Perplexity')
    axes2[0].set_ylabel('Trustworthiness')
    axes2[0].set_title('Trustworthiness vs Perplexity')
    axes2[0].grid(True, alpha=0.3)
    
    # Final cost vs perplexity
    axes2[1].plot(perps, costs, 'o-', linewidth=2, markersize=8, color='red')
    axes2[1].set_xlabel('Perplexity')
    axes2[1].set_ylabel('Final KL Divergence')
    axes2[1].set_title('Final Cost vs Perplexity')
    axes2[1].grid(True, alpha=0.3)
    
    # Cost evolution for different perplexities
    for perp in [5, 30, 100]:
        costs_evolution = results[perp]['tsne'].costs
        axes2[2].plot(costs_evolution, label=f'Perplexity {perp}', alpha=0.8)
    
    axes2[2].set_xlabel('Iteration')
    axes2[2].set_ylabel('KL Divergence')
    axes2[2].set_title('Cost Evolution')
    axes2[2].legend()
    axes2[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return results

# Run perplexity analysis
print("=== Perplexity Analysis ===")
perplexity_results = analyze_perplexity_effects()

print("\\n=== Perplexity Results Summary ===")
for perp, result in perplexity_results.items():
    print(f"\\nPerplexity {perp}:")
    print(f"  Trustworthiness: {result['trustworthiness']:.4f}")
    print(f"  Final Cost: {result['final_cost']:.4f}")
    print(f"  Convergence iterations: {len(result['tsne'].costs)}")


In [ ]:
# Compare early exaggeration effects
def analyze_early_exaggeration():
    """Analyze the effects of early exaggeration on t-SNE convergence"""
    
    # Create a challenging dataset (Swiss roll)
    X, color = make_swiss_roll(n_samples=300, noise=0.1, random_state=42)
    
    # Standardize
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Test different early exaggeration values
    exaggeration_values = [1, 4, 12, 20, 50]  # 1 means no exaggeration
    
    results = {}
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for i, exag in enumerate(exaggeration_values):
        print(f"\\nTesting early exaggeration = {exag}")
        
        # Fit t-SNE with different exaggeration
        tsne = AdvancedTSNE(perplexity=30, n_iter=600, early_exaggeration=exag, learning_rate=200)
        Y = tsne.fit_transform(X_scaled)
        
        # Calculate metrics
        trust_score = trustworthiness(X_scaled, Y, n_neighbors=10)
        final_cost = tsne.costs[-1]
        
        results[exag] = {
            'embedding': Y,
            'trustworthiness': trust_score,
            'final_cost': final_cost,
            'costs': tsne.costs,
            'grad_norms': tsne.grad_norms
        }
        
        # Plot embedding
        scatter = axes[i].scatter(Y[:, 0], Y[:, 1], c=color, cmap='viridis', alpha=0.7)
        axes[i].set_title(f'Early Exag = {exag}\\nTrust: {trust_score:.3f}')
        axes[i].set_xlabel('t-SNE 1')
        axes[i].set_ylabel('t-SNE 2')
        
        if i >= len(exaggeration_values):
            axes[i].set_visible(False)
    
    # Additional analysis plots
    fig2, axes2 = plt.subplots(2, 2, figsize=(15, 10))
    
    # Cost evolution
    axes2[0, 0].set_title('Cost Evolution')
    for exag in exaggeration_values:
        axes2[0, 0].plot(results[exag]['costs'], label=f'Exag {exag}', alpha=0.8)
    axes2[0, 0].set_xlabel('Iteration')
    axes2[0, 0].set_ylabel('KL Divergence')
    axes2[0, 0].legend()
    axes2[0, 0].grid(True, alpha=0.3)
    
    # Gradient norm evolution
    axes2[0, 1].set_title('Gradient Norm Evolution')
    for exag in exaggeration_values:
        grad_norms = results[exag]['grad_norms']
        axes2[0, 1].plot(grad_norms, label=f'Exag {exag}', alpha=0.8)
    axes2[0, 1].set_xlabel('Iteration')
    axes2[0, 1].set_ylabel('Gradient Norm')
    axes2[0, 1].set_yscale('log')
    axes2[0, 1].legend()
    axes2[0, 1].grid(True, alpha=0.3)
    
    # Final metrics comparison
    exag_vals = list(results.keys())
    trusts = [results[e]['trustworthiness'] for e in exag_vals]
    costs = [results[e]['final_cost'] for e in exag_vals]
    
    axes2[1, 0].bar(range(len(exag_vals)), trusts, alpha=0.7, color='skyblue')
    axes2[1, 0].set_xlabel('Early Exaggeration')
    axes2[1, 0].set_ylabel('Trustworthiness')
    axes2[1, 0].set_title('Final Trustworthiness')
    axes2[1, 0].set_xticks(range(len(exag_vals)))
    axes2[1, 0].set_xticklabels(exag_vals)
    
    # Add values on bars
    for i, trust in enumerate(trusts):
        axes2[1, 0].text(i, trust + 0.01, f'{trust:.3f}', ha='center')
    
    axes2[1, 1].bar(range(len(exag_vals)), costs, alpha=0.7, color='lightcoral')
    axes2[1, 1].set_xlabel('Early Exaggeration')
    axes2[1, 1].set_ylabel('Final KL Divergence')
    axes2[1, 1].set_title('Final Cost')
    axes2[1, 1].set_xticks(range(len(exag_vals)))
    axes2[1, 1].set_xticklabels(exag_vals)
    
    # Add values on bars
    for i, cost in enumerate(costs):
        axes2[1, 1].text(i, cost + 0.05, f'{cost:.2f}', ha='center')
    
    plt.tight_layout()
    plt.show()
    
    return results

# Run early exaggeration analysis
print("=== Early Exaggeration Analysis ===")
exaggeration_results = analyze_early_exaggeration()

print("\\n=== Early Exaggeration Results Summary ===")
for exag, result in exaggeration_results.items():
    print(f"\\nEarly Exaggeration {exag}:")
    print(f"  Trustworthiness: {result['trustworthiness']:.4f}")
    print(f"  Final Cost: {result['final_cost']:.4f}")
    print(f"  Final Gradient Norm: {result['grad_norms'][-1]:.6f}")
    
    # Analyze convergence speed
    costs = np.array(result['costs'])
    initial_cost = costs[0]
    final_cost = costs[-1]
    improvement = initial_cost - final_cost
    print(f"  Total Improvement: {improvement:.4f}")
    print(f"  Convergence Rate: {improvement / len(costs):.6f} per iteration")


In [ ]:
# Demonstrate the crowding problem and how different distributions address it
def demonstrate_crowding_problem():
    """Demonstrate the crowding problem and distribution effects"""
    
    # Create high-dimensional dataset with known structure
    np.random.seed(42)
    n_clusters = 5
    n_points_per_cluster = 50
    dims = 100
    
    # Generate clusters in high-dimensional space
    X = []
    y = []
    cluster_centers = np.random.randn(n_clusters, dims) * 3
    
    for i in range(n_clusters):
        cluster_points = cluster_centers[i] + np.random.randn(n_points_per_cluster, dims) * 0.5
        X.append(cluster_points)
        y.extend([i] * n_points_per_cluster)
    
    X = np.vstack(X)
    y = np.array(y)
    
    # Standardize
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    print(f"Dataset: {X_scaled.shape[0]} points in {X_scaled.shape[1]} dimensions")
    
    # Analyze distance distributions in high-dimensional space
    distances_hd = []
    n_samples = X_scaled.shape[0]
    
    # Sample distances to avoid memory issues
    sample_size = min(1000, n_samples // 2)
    indices = np.random.choice(n_samples, sample_size, replace=False)
    
    for i in indices:
        for j in range(i+1, min(i+50, n_samples)):  # Limit to avoid too many distances
            dist = np.linalg.norm(X_scaled[i] - X_scaled[j])
            distances_hd.append(dist)
    
    distances_hd = np.array(distances_hd)
    
    # Compare different embedding methods
    methods = {
        'PCA': PCA(n_components=2),
        't-SNE (Gaussian)': 't-sne-gaussian',  # We'll implement this
        't-SNE (Student-t)': TSNE(n_components=2, perplexity=30, random_state=42)
    }
    
    embeddings = {}
    
    # Fit embeddings
    for name, method in methods.items():
        print(f"\\nFitting {name}...")
        
        if name == 'PCA':
            embedding = method.fit_transform(X_scaled)
        elif name == 't-SNE (Gaussian)':
            # We'll use a modified version that uses Gaussian in low-dim space
            embedding = fit_tsne_gaussian(X_scaled)
        else:
            embedding = method.fit_transform(X_scaled)
        
        embeddings[name] = embedding
    
    # Analyze distance preservation
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    
    # Plot embeddings
    for i, (name, embedding) in enumerate(embeddings.items()):
        ax = axes[0, i]
        scatter = ax.scatter(embedding[:, 0], embedding[:, 1], c=y, cmap='tab10', alpha=0.7)
        ax.set_title(f'{name}')
        ax.set_xlabel('Component 1')
        ax.set_ylabel('Component 2')
    
    # Plot distance distributions
    ax_dist = axes[0, 3]
    ax_dist.hist(distances_hd, bins=50, alpha=0.7, density=True, label='High-dim distances')
    ax_dist.set_xlabel('Distance')
    ax_dist.set_ylabel('Density')
    ax_dist.set_title('High-Dim Distance Distribution')
    ax_dist.legend()
    
    # Analyze distance preservation for each method
    for i, (name, embedding) in enumerate(embeddings.items()):
        ax = axes[1, i]
        
        # Compute distances in embedding space
        distances_ld = []
        for idx1 in indices[:min(len(indices), 100)]:  # Limit for computation
            for idx2 in range(idx1+1, min(idx1+50, n_samples)):
                if idx2 < len(embedding):
                    dist = np.linalg.norm(embedding[idx1] - embedding[idx2])
                    distances_ld.append(dist)
        
        distances_ld = np.array(distances_ld)
        
        # Plot distance comparison
        ax.scatter(distances_hd[:len(distances_ld)], distances_ld, alpha=0.3)
        ax.plot([0, distances_hd.max()], [0, distances_hd.max()], 'r--', alpha=0.5)
        ax.set_xlabel('High-dim Distance')
        ax.set_ylabel('Low-dim Distance')
        ax.set_title(f'{name}\\nDistance Preservation')
        
        # Calculate correlation
        if len(distances_ld) > 0:
            corr = np.corrcoef(distances_hd[:len(distances_ld)], distances_ld)[0, 1]
            ax.text(0.05, 0.95, f'Corr: {corr:.3f}', transform=ax.transAxes, 
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Distance distribution comparison
    ax_compare = axes[1, 3]
    
    # Plot Gaussian vs Student-t distributions
    x = np.linspace(-4, 4, 1000)
    gaussian = (1/np.sqrt(2*np.pi)) * np.exp(-0.5 * x**2)
    student_t = (1/np.pi) * (1 / (1 + x**2))  # Cauchy distribution (t with 1 df)
    
    ax_compare.plot(x, gaussian, label='Gaussian', linewidth=2)
    ax_compare.plot(x, student_t, label='Student-t (df=1)', linewidth=2)
    ax_compare.set_xlabel('Value')
    ax_compare.set_ylabel('Probability Density')
    ax_compare.set_title('Distribution Comparison')
    ax_compare.legend()
    ax_compare.set_yscale('log')
    ax_compare.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print analysis
    print("\\n=== Crowding Problem Analysis ===")
    print(f"High-dimensional distance statistics:")
    print(f"  Mean: {np.mean(distances_hd):.4f}")
    print(f"  Std: {np.std(distances_hd):.4f}")
    print(f"  Min: {np.min(distances_hd):.4f}")
    print(f"  Max: {np.max(distances_hd):.4f}")
    
    print("\\nKey Insights:")
    print("- High-dimensional distances become more uniform (curse of dimensionality)")
    print("- Student-t distribution has heavier tails than Gaussian")
    print("- Heavy tails allow distant points to be placed further apart in 2D")
    print("- This helps address the crowding problem in t-SNE")
    
    return embeddings, distances_hd

def fit_tsne_gaussian(X, perplexity=30, n_iter=300):
    """Simplified t-SNE using Gaussian distribution in low-dimensional space"""
    n = X.shape[0]
    
    # Compute high-dimensional probabilities (same as regular t-SNE)
    distances = np.sum((X[:, np.newaxis] - X[np.newaxis, :]) ** 2, axis=2)
    np.fill_diagonal(distances, np.inf)
    
    # Simple approach: use fixed sigma for all points
    sigma = np.sqrt(perplexity / 2)
    P = np.exp(-distances / (2 * sigma**2))
    np.fill_diagonal(P, 0)
    P = P / np.sum(P, axis=1, keepdims=True)
    P = (P + P.T) / (2 * n)
    P = np.maximum(P, 1e-12)
    
    # Initialize embedding
    Y = np.random.normal(0, 1e-4, (n, 2))
    
    # Optimize using Gaussian distribution in low-dimensional space
    for i in range(n_iter):
        # Compute low-dimensional probabilities using Gaussian
        distances_ld = np.sum((Y[:, np.newaxis] - Y[np.newaxis, :]) ** 2, axis=2)
        Q = np.exp(-distances_ld)
        np.fill_diagonal(Q, 0)
        Q = Q / np.sum(Q)
        Q = np.maximum(Q, 1e-12)
        
        # Compute gradient
        PQ_diff = P - Q
        gradient = np.zeros_like(Y)
        
        for j in range(n):
            diff = Y[j] - Y
            gradient[j] = 4 * np.sum((PQ_diff[j] * np.exp(-distances_ld[j]))[:, np.newaxis] * diff, axis=0)
        
        # Update with simple gradient descent
        Y -= 100 * gradient
        Y = Y - np.mean(Y, axis=0)  # Center
    
    return Y

# Run crowding problem demonstration
print("=== Crowding Problem Demonstration ===")
crowding_results, hd_distances = demonstrate_crowding_problem()
